In [3]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [6]:
dog_data = pd.read_csv('NYC_Dog_Licensing_Dataset_20241105.csv')
dog_data.head()

C:\Users\cshar\AppData\Local\Temp\ipykernel_22780\421662572.py:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  dog_data = pd.read_csv('NYC_Dog_Licensing_Dataset_20241105.csv')


,AnimalName,AnimalGender,AnimalBirthYear,BreedName,ZipCode,LicenseIssuedDate,LicenseExpiredDate,Extract Year
0,PAIGE,F,2014,American Pit Bull Mix / Pit Bull Mix,10035.0,09-12-2014,09-12-2017,2016
1,YOGI,M,2010,Boxer,10465.0,09-12-2014,10-02-2017,2016
2,ALI,M,2014,Basenji,10013.0,09-12-2014,09-12-2019,2016
3,QUEEN,F,2013,Akita Crossbreed,10013.0,09-12-2014,09-12-2017,2016
4,LOLA,F,2009,Maltese,10028.0,09-12-2014,10-09-2017,2016


In [9]:
stats = dog_data.describe()
stats

,ZipCode,Extract Year
count,616881.000000,616890.000000
mean,10704.959263,2020.018499
std,1086.502249,2.769779
min,0.000000,2016.000000
25%,10031.000000,2017.000000
50%,10469.000000,2022.000000
75%,11228.000000,2022.000000
max,99508.000000,2023.000000
